In [2]:
import pandas as pd

# load the full VIIRS fire dataset
fires_df = pd.read_csv("../data/viirs_fires_italy.csv")

# bounding box for agrimonia, italy
lat_min, lat_max = 45.1678524, 47.1678524
lon_min, lon_max = 8.87920992, 10.87920992

# filter by bounding box
fires_italy = fires_df[
    (fires_df["latitude"].between(lat_min, lat_max)) &
    (fires_df["longitude"].between(lon_min, lon_max))
].copy()

# convert acquisition date to datetime
fires_italy["date"] = pd.to_datetime(fires_italy["acq_date"], format="%Y-%m-%d")

# aggregate to daily summary
fires_daily = (
    fires_italy
    .groupby("date")
    .agg(
        fire_count = ("frp", "size"),      # number of fire pixels
        avg_frp    = ("frp", "mean")       # mean fire radiative power
    )
    .reset_index()
)

# build full daily index for time span
full_idx = pd.DataFrame({
    "date": pd.date_range("2016-01-01", "2021-12-31", freq="D")
})

# fill missing days with zeros
fires_complete = (
    full_idx
    .merge(fires_daily, on="date", how="left")
    .assign(
        fire_count = lambda df: df["fire_count"].fillna(0).astype(int),
        avg_frp    = lambda df: df["avg_frp"].fillna(0.0)
    )
)

print(f" Daily summary produced: {len(fires_complete)} days (should be 365)")
print(f" Missing days filled: {(fires_complete['fire_count']==0).sum()} days")

# save cleaned daily summary
fires_complete.to_csv("../data/agrimonia_fire_counts_daily.csv", index=False)
print(" Saved ../data/agrimonia_fire_counts_daily.csv.csv")


 Daily summary produced: 2192 days (should be 365)
 Missing days filled: 615 days
 Saved ../data/agrimonia_fire_counts_daily.csv.csv
